# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
display(df.head(5))
print()
df.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Всего 21525 записей (строк). В столбцах: 'days_employed' и 'total_income' имеются пропуски. в столбце - 'days_employed' имеются отрицательные значения и тип данных float. данные пропуски могут возникнуть из-за того либо клиент не указал данные, либо ошибка при выгрузке. Отрицательные значения в столбце - 'days_employed' появились из-за некарректного исчисления стажа.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
#Провера пропущеных знчений
print(df.isna().sum())
print()
# заполнение пропущенныхсначением -0
df = df.fillna(0)
print()
# проверка
print(df.isna().sum())
print()
# удаление пропущенных значений
df.dropna()
print()
# изучаю униакльные значения по конретным столбцам
print(df['income_type'].unique())
print()
print(df['education'].unique())
print()
print(df['family_status'].unique())
print()
#подсчитываю количество по категориям в столбце incom_type
print('Количество сотрудников:', df[df['income_type']== 'сотрудник']['income_type'].count())
print('Количество пенсионеров:', df[df['income_type']== 'пенсионер']['income_type'].count())
print('Количество компаньонов:', df[df['income_type']== 'компаньон']['income_type'].count())
print('Количество госслужащих:', df[df['income_type']== 'госслужащий']['income_type'].count())
print('Количество безработных:', df[df['income_type']== 'безработный']['income_type'].count())
print('Количество предпринимателей:', df[df['income_type']== 'предприниматель']['income_type'].count())
print('Количество студентов:', df[df['income_type']== 'студент']['income_type'].count())
print('Количество в декрете:', df[df['income_type']== 'в декрете']['income_type'].count())
print()


# узнаю уникальные значения в столбце children
print(df['children'].unique())
print()
print('Процент строк с некорректными данными количества детей: {:.2%}'.format((df[df['children'] == -1]['children'].count() + df[df['children'] == 20]['children'].count()) / df['children'].count()))
print()

#удаляю строки с некорректными данными
df = df[df['children'] != 20]
df = df[df['children'] != -1]
#повторно проверяю уникальные значения в столбце children
print(df['children'].unique())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']

['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']

['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не

**Вывод**

В таблицах: 'days_employed' и 'total_income' имеются пропуски в колчестве 2174 строк, которые заполнили - 0.вывил все уникальные знаения по конкретным столбцам. Проверил остались ли пропуски. Подсчитал и вывел для просмотра все категории по столбцу 'income-type'. обнаружил отрицатеьные значения детей и  сильный выброс количества детей у клиентов. тк  их общее коичество составляет 0.57% - то эти значения можно не учитывать

### Замена типа данных

In [3]:
# тк данные  представлены в виде рациональных чисел использую данный метод изменения типа данных в целочисленный
df = df.astype({'days_employed': int, 'total_income': int})
display(df.head(5)) #проверил изменение типа данных в столбцах 'days_employed' и 'total_income'
df.dtypes

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

**Вывод**

изменил типы данных на целочисленный и вывел информацию о типах данных в таблице

### Обработка дубликатов

In [4]:
# привел все значения в колоннке 'education' к одному регистру
df['education'] = df['education'].str.lower()
# проверил уникальные значения в колонке 'education'
print(df['education'].unique())
print()
print()
#посчитал количество дубликатов
print(df.duplicated().sum())
print()
# удалил все дубликаты
df = df.drop_duplicates().reset_index(drop = True)
print(df.duplicated().sum())
print()
#  отфильтровал всех по категориям 'total_income'  и заполнил все нулевые значения - медианными знаениями
median_income_emploee = df[df['income_type'] == 'сотрудник']['total_income'].median()
print('медианный доход сотрудников:', median_income_emploee)
median_income_retiree = df[df['income_type'] == 'пенсионер']['total_income'].median()
print('медианный доход пенсионеров:', median_income_retiree)
median_income_companion = df[df['income_type'] == 'компаньон']['total_income'].median()
print('медианный доход компаньонов:', median_income_companion)
median_income_sivil_servant = df[df['income_type'] == 'госслужащий']['total_income'].median()
print('медианный доход госслужащего:', median_income_sivil_servant)
print()
print()
print('считаем пустые значения в столбце total_income по отфильтрованным сотрудникам:', df[(df['income_type'] == 'сотрудник') & (df['total_income'] == 0)]['income_type'].count())
print('считаем пустые значения в столбце total_income по отфильтрованным пенсионерам:', df[(df['income_type'] == 'пенсионер') & (df['total_income'] == 0)]['income_type'].count())
print('считаем пустые значения в столбце total_income по отфильтрованным компаньонам:', df[(df['income_type'] == 'компаньон') & (df['total_income'] == 0)]['income_type'].count())
print('считаем пустые значения в столбце total_income  по отфильтрованным госслужащим:', df[(df['income_type'] == 'госслужащий') & (df['total_income'] == 0)]['income_type'].count())
print()
print()
# заменяю все нулевые значения намедианные
df.loc[((df['income_type'] == 'сотрудник') & (df['total_income'] == 0)), 'total_income'] = median_income_emploee 
df.loc[((df['income_type'] == 'пенсионер') & (df['total_income'] == 0)), 'total_income'] = median_income_emploee
df.loc[((df['income_type'] == 'компаньон') & (df['total_income'] == 0)), 'total_income'] = median_income_emploee
df.loc[((df['income_type'] == 'госслужащий') & (df['total_income'] == 0)), 'total_income'] = median_income_emploee

print('перепроверка:')
print(df[(df['income_type'] == 'сотрудник') & (df['total_income'] == 0)]['income_type'].count())
print(df[(df['income_type'] == 'пенсионер') & (df['total_income'] == 0)]['income_type'].count())
print(df[(df['income_type'] == 'компаньон') & (df['total_income'] == 0)]['income_type'].count())
print(df[(df['income_type'] == 'госслужащий') & (df['total_income'] == 0)]['income_type'].count())


['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


71

0

медианный доход сотрудников: 133885.0
медианный доход пенсионеров: 110779.5
медианный доход компаньонов: 162513.0
медианный доход госслужащего: 139304.0


считаем пустые значения в столбце total_income по отфильтрованным сотрудникам: 1066
считаем пустые значения в столбце total_income по отфильтрованным пенсионерам: 383
считаем пустые значения в столбце total_income по отфильтрованным компаньонам: 497
считаем пустые значения в столбце total_income  по отфильтрованным госслужащим: 144


перепроверка:
0
0
0
0


**Вывод**

Проверил наличие дубликатов. Удалил дубликаты и сбросил индексацию. Убедился что дубликатов нет.Посчитал медианный доход для всех котегорий и заполнил нулевые котегории медианным значением дохода.
Дубликаты могли возникнуть из за того что данные были представлены в разных регистрах, ошибок при заполнении датасети и/или при выгрузке.

нулевое значение в income_type у одного предпринемателя из 2-х не повредит анализу, по этому не заполняюю это нулевое значение.

### Лемматизация

In [12]:
#привожу все значения к одному регистру
df['purpose'] = df['purpose'].str.lower()
# вызываю стеммер для приведения всех слов к лемме
from pymystem3 import Mystem
m = Mystem()
# создаю список уникальных значений по столбцу 'purpose'
purpose_list = df['purpose'].unique()
#создаю пусто список лемм куда буду добавлять леммы через цикл
lemmas = []
for i in purpose_list:
    lemma = m.lemmatize(i)
    lemmas.extend(lemma)
print(lemmas)

# создаю цикл для поиска конкретных лемм и добавленя в столбец 'lemmatize_purpose'
def lemmas_purpose(row):
    lemmas_row = m.lemmatize(row)
    
    if 'автомобиль' in lemmas_row:
         return 'автомобиль'
    elif 'свадьба' in lemmas_row:
        return  'свадьба'
    elif 'образование' in lemmas_row:
        return 'образование'
    elif 'ремонт' in lemmas_row:
        return 'ремонт'
    elif 'недвижимость' or 'жилье' in lemmas_row:
        return  'недвижимость'
    
    return

df['lemmatize_purpose']= df['purpose'].apply(lemmas_purpose)
display(df.tail(10))


['покупка', ' ', 'жилье', '\n', 'приобретение', ' ', 'автомобиль', '\n', 'дополнительный', ' ', 'образование', '\n', 'сыграть', ' ', 'свадьба', '\n', 'операция', ' ', 'с', ' ', 'жилье', '\n', 'образование', '\n', 'на', ' ', 'проведение', ' ', 'свадьба', '\n', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n', 'покупка', ' ', 'недвижимость', '\n', 'покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n', 'покупка', ' ', 'жилой', ' ', 'недвижимость', '\n', 'строительство', ' ', 'собственный', ' ', 'недвижимость', '\n', 'недвижимость', '\n', 'строительство', ' ', 'недвижимость', '\n', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n', 'на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n', 'операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n', 'строительство', ' ', 'жилой', ' ', 'недвижимость', '\n', 'жилье', '\n', 'операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n', 'автомобиль', '\n', 'заниматься', ' ', 'образование', '\n', 'сделка', ' '

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmatize_purpose,categ_income
21321,1,-467,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.0,заняться образованием,образование,средний доход
21322,0,-914,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.0,покупка своего жилья,недвижимость,средний доход
21323,0,-404,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.0,на покупку своего автомобиля,автомобиль,средний доход
21324,0,373995,59,среднее,1,женат / замужем,0,F,пенсионер,0,153864.0,сделка с автомобилем,автомобиль,средний доход
21325,1,-2351,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.0,покупка коммерческой недвижимости,недвижимость,средний доход
21326,1,-4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.0,операции с жильем,недвижимость,средний доход
21327,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.0,сделка с автомобилем,автомобиль,средний доход
21328,1,-2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.0,недвижимость,недвижимость,низкий доход
21329,3,-3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.0,на покупку своего автомобиля,автомобиль,средний доход
21330,2,-1984,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.0,на покупку автомобиля,автомобиль,низкий доход


**Вывод**

Провел лемматизацию столбца 'purpose' и выявил конкретные цели кредитования для дальнейшего анализа

### Категоризация данных

In [6]:
# нахожу мин,макс,среднее,медианное значения доходов
print(df['total_income'].max())
print(df['total_income'].min())
print(int(df['total_income'].mean()))
print(df['total_income'].median())

#создаю функцию для  категоризации данных по доходам
def categ_income(row):
    if row < 100000:
        return 'низкий доход'
    elif row < 350000:
        return 'средний доход'
    else:
        return 'высокий доход'
    
df['categ_income'] = df['total_income'].apply(categ_income)
display(df.head(5))

education_dict = {0:"высшее", 1:"среднее", 2:"неоконченное высшее", 3:"начальное", 4:"ученая степень"}
print(education_dict)

family_status_dict = {0:"женат/замужем", 1:"гражданский брак", 2:"вдовец/вдова", 3:"вразводе", 4:"не женат/не замужем"}
print(family_status_dict)

2265604.0
0.0
164151
135751.0


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmatize_purpose,categ_income
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.0,покупка жилья,недвижимость,средний доход
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.0,приобретение автомобиля,автомобиль,средний доход
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.0,покупка жилья,недвижимость,средний доход
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.0,дополнительное образование,образование,средний доход
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.0,сыграть свадьбу,свадьба,средний доход


{0: 'высшее', 1: 'среднее', 2: 'неоконченное высшее', 3: 'начальное', 4: 'ученая степень'}
{0: 'женат/замужем', 1: 'гражданский брак', 2: 'вдовец/вдова', 3: 'вразводе', 4: 'не женат/не замужем'}


In [7]:
df['categ_income'].value_counts()

средний доход    16030
низкий доход      4445
высокий доход      856
Name: categ_income, dtype: int64

**Вывод**

Категоризировал зданные при помощи функций. Выделил категории по доходу

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [8]:
count_without_children = (df[(df['debt'] == 1) & (df['children'] == 0)].count() / df[df['debt'] == 0]['children'].count())['debt']
print('Процент невозврата кредитов среди тех у кого нет детей: {:.0f} % '.format(count_without_children * 100))

count_without_children = (df[(df['debt'] == 1) & (df['children'] == 1)].count() / df[df['debt'] == 0]['children'].count())['debt']
print('Процент невозврата кредитов среди тех у кого 1 ребенок: {:.0f} % '.format(count_without_children * 100))
count_without_children = (df[(df['debt'] == 1) & (df['children'] == 2)].count() / df[df['debt'] == 0]['children'].count())['debt']
print('Процент невозврата кредитов среди тех у кого 2 ребенка: {:.0f} % '.format(count_without_children * 100))
count_without_children = (df[(df['debt'] == 1) & (df['children'] == 3)].count() / df[df['debt'] == 0]['children'].count())['debt']
print('Процент невозврата кредитов среди тех у кого 3 ребенка: {:.0f} % '.format(count_without_children * 100))
count_without_children = (df[(df['debt'] == 1) & (df['children'] == 4)].count() / df[df['debt'] == 0]['children'].count())['debt']
print('Процент невозврата кредитов среди тех у кого 4 ребенка: {:.0f} % '.format(count_without_children * 100))
count_without_children = (df[(df['debt'] == 1) & (df['children'] == 5)].count() / df[df['debt'] == 0]['children'].count())['debt']
print('Процент невозврата кредитов среди тех у кого 5 детей: {:.0f} % '.format(count_without_children * 100))



Процент невозврата кредитов среди тех у кого нет детей: 5 % 
Процент невозврата кредитов среди тех у кого 1 ребенок: 2 % 
Процент невозврата кредитов среди тех у кого 2 ребенка: 1 % 
Процент невозврата кредитов среди тех у кого 3 ребенка: 0 % 
Процент невозврата кредитов среди тех у кого 4 ребенка: 0 % 
Процент невозврата кредитов среди тех у кого 5 детей: 0 % 


**Вывод**

Т.к. те у кого нет детей возвращают на 2% реже чем те у кого есть хоть 1 ребенок, значит наличие детей не влияет на возврат кредитов.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [9]:
print(df.groupby('family_status')['debt'].value_counts())


count_family_status_not_married = df.groupby('family_status').agg({'debt': ['mean']})
print(count_family_status_not_married)

family_status          debt
Не женат / не замужем  0        2523
                       1         273
в разводе              0        1105
                       1          84
вдовец / вдова         0         888
                       1          63
гражданский брак       0        3749
                       1         385
женат / замужем        0       11334
                       1         927
Name: debt, dtype: int64
                           debt
                           mean
family_status                  
Не женат / не замужем  0.097639
в разводе              0.070648
вдовец / вдова         0.066246
гражданский брак       0.093130
женат / замужем        0.075606


**Вывод**

те кто в разводе или вдовцы/вдовы возвращают кредиты чаще как женатые/замужем  и в разводе. В то время  как не женатые/не замужем и в гражданском браке возвращают реже

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [10]:
print(df.groupby('categ_income')['debt'].value_counts())

count_income_low = (df[(df['debt'] == 1) & (df['categ_income'] == 'низкий доход')].count() / df[df['categ_income'] == 'низкий доход']['debt'].count())['debt']
print('Процент невозврата кредитов среди тех у кто низкий доход: {:.0f} % '.format(count_income_low * 100))

count_income_average = (df[(df['debt'] == 1) & (df['categ_income'] == 'средний доход')].count() / df[df['categ_income'] == 'средний доход']['debt'].count())['debt']
print('Процент невозврата кредитов среди тех у кто  средний доход: {:.0f} % '.format(count_income_average * 100))

count_income_high = (df[(df['debt'] == 1) & (df['categ_income'] == 'высокий доход')].count() / df[df['categ_income'] == 'высокий доход']['debt'].count())['debt']
print('Процент невозврата кредитов среди тех у кто высокий доход: {:.0f} % '.format(count_income_high * 100))

categ_income   debt
высокий доход  0         801
               1          55
низкий доход   0        4091
               1         354
средний доход  0       14707
               1        1323
Name: debt, dtype: int64
Процент невозврата кредитов среди тех у кто низкий доход: 8 % 
Процент невозврата кредитов среди тех у кто  средний доход: 8 % 
Процент невозврата кредитов среди тех у кто высокий доход: 6 % 


**Вывод**

Люди с высоким доходом возвращают кредиты чаще

- Как разные цели кредита влияют на его возврат в срок?

In [11]:
print(df.groupby('lemmatize_purpose')['debt'].value_counts())


data_pivot = df.pivot_table(index = ['lemmatize_purpose'], values = 'debt', aggfunc = 'mean')
print(data_pivot)

lemmatize_purpose  debt
автомобиль         0       3879
                   1        400
недвижимость       0       9402
                   1        745
образование        0       3619
                   1        369
ремонт             0        569
                   1         35
свадьба            0       2130
                   1        183
Name: debt, dtype: int64
                       debt
lemmatize_purpose          
автомобиль         0.093480
недвижимость       0.073421
образование        0.092528
ремонт             0.057947
свадьба            0.079118


**Вывод**

Чаще всего кредиты возвращает те кто берет деньги на ремонт и покупку недвижимости

Вопрос: а как выводить числовые знаения - целыми?

## Шаг 4. Общий вывод

При исследовании надежности заемщиков изучил общую информацию датасета. Заполнил пустые значеня и удалил дубликаты. Привел все данные к одному регистру, выделил леммы для категоризации целей кредита, разделил заемщиков на категории по доходу.

Исходя из анализа данных пришел к выводу  что на возврат кредитов влияют:
 количество детей, тк заемщики без детей на 2% реже возвращают кредит
 семейное поожение - заемщики которые не состот вотношениях возвращают кредиты  на 2%чаще
 цели кридита связанные с недвижимостью и ремонтом возвращаются чаще на 2-3%
 доход заемщика - чем выше зарплата зарплате тем чаще возращают кредит
 
 В конце исследования ответил на поставленные вопросы и сделал выводы.